# Forward-mode automatic differentiation for Linear Mixed Models

In this notebook we apply the formulas derived in [Differentiation of the Cholesky decomposition](https://arxiv.org/abs/1602.07527) to the evaluation of the profiled log-likelihood of a linear mixed model.

## Examples of linear mixed models

### A model with a single, simple, scalar random-effects term

Consider first a mixed-effects model with a single, simple, scalar random-effects term.

First, load the packages to be used

In [1]:
using DataFrames, LinearAlgebra, MixedModels, Statistics
using MixedModels: dataset

┌ Info: Precompiling MixedModels [ff71e718-51f3-5ec2-a782-8ffcbfa3c316]
└ @ Base loading.jl:1260


and load the `dyestuff` data set

In [2]:
dyestuff = dataset(:dyestuff)
describe(dyestuff)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Nothing,DataType
1,batch,,A,,F,6,,String
2,yield,1527.5,1440,1530.0,1635,,,Int16


There are a total of 30 measurements of the `yield`, 5 each for the 6 different batches.

Fit a model with an overall `(Intercept)`, an estimate of the mean response when the `batch` is not specified, and random effects for each level of `batch`.

In [3]:
fm1 = fit(MixedModel, @formula(yield ~ 1+(1|batch)), dyestuff)

Linear mixed model fit by maximum likelihood
 yield ~ 1 + (1 | batch)
   logLik   -2 logLik     AIC        BIC    
 -163.66353  327.32706  333.32706  337.53065

Variance components:
            Column    Variance  Std.Dev. 
batch    (Intercept)  1388.3333 37.260345
Residual              2451.2500 49.510100
 Number of obs: 30; levels of grouping factors: 6

  Fixed-effects parameters:
──────────────────────────────────────────────────
             Estimate  Std.Error  z value  P(>|z|)
──────────────────────────────────────────────────
(Intercept)    1527.5    17.6946    86.33   <1e-99
──────────────────────────────────────────────────

The _maximum likelihood_ parameter estimates, $\hat\beta=1527.5$, $\hat{\sigma_1}=37.260$, and $\hat{\sigma}=49.510$, are the values that minimize the objective function, negative twice the log-likelihood.

In [4]:
objective(fm1)

327.3270598811401

The optimization of the objective is performed indirectly using the `profiled log-likelihood` that depends only on the ratio $\theta=\sigma_1/\sigma$, evaluated from `L` the (lower) Cholesky factor of an augmented cross-product matrix.

In [5]:
fm1.θ

1-element Array{Float64,1}:
 0.7525806757718846

In [6]:
LowerTriangular(fm1.L)

8×8 LowerTriangular{Float64,BlockArrays.BlockArray{Float64,2,Array{AbstractArray{Float64,2},2},Tuple{BlockArrays.BlockedUnitRange{Array{Int64,1}},BlockArrays.BlockedUnitRange{Array{Int64,1}}}}} with indices 1:1:8×1:1:8:
    1.95752      ⋅           ⋅       …      ⋅           ⋅          ⋅   
    0.0         1.95752      ⋅              ⋅           ⋅          ⋅   
    0.0         0.0         1.95752         ⋅           ⋅          ⋅   
    0.0         0.0         0.0             ⋅           ⋅          ⋅   
    0.0         0.0         0.0             ⋅           ⋅          ⋅   
    0.0         0.0         0.0    
 ──────────────────────────────────  …     1.95752      ⋅          ⋅   
 ───────────────────────────────
    1.92228     1.92228     1.92228
 ──────────────────────────────────        1.92228     2.79804     ⋅   
 ───────────────────────────────
 2893.03     2937.24     3006.45        2825.75     4274.01     271.178

In practice the matrix `L` is evaluated and stored as a `BlockMatrix`, as shown here, allowing for simplifications from sparsity in the blocks.  For example, the upper-left $6\times 6$ block is diagonal. 

In [7]:
fm1.L[Block(1,1)]

6×6 Diagonal{Float64,Array{Float64,1}}:
 1.95752   ⋅        ⋅        ⋅        ⋅        ⋅ 
  ⋅       1.95752   ⋅        ⋅        ⋅        ⋅ 
  ⋅        ⋅       1.95752   ⋅        ⋅        ⋅ 
  ⋅        ⋅        ⋅       1.95752   ⋅        ⋅ 
  ⋅        ⋅        ⋅        ⋅       1.95752   ⋅ 
  ⋅        ⋅        ⋅        ⋅        ⋅       1.95752

In deriving the profiled log-likelihood and its gradient it simplest to start with general matrices and consider adjustments for blocks later.  The `copyltri` utility extracts the lower triangle as a `Matrix`.

In [8]:
L = MixedModels.copyltri(fm1.L)

8×8 Array{Float64,2}:
    1.95752     0.0         0.0      …     0.0         0.0        0.0
    0.0         1.95752     0.0            0.0         0.0        0.0
    0.0         0.0         1.95752        0.0         0.0        0.0
    0.0         0.0         0.0            0.0         0.0        0.0
    0.0         0.0         0.0            0.0         0.0        0.0
    0.0         0.0         0.0      …     1.95752     0.0        0.0
    1.92228     1.92228     1.92228        1.92228     2.79804    0.0
 2893.03     2937.24     3006.45        2825.75     4274.01     271.178

The original data provides a symmetric blocked matrix, `A`, from which `L` is evaluated

In [9]:
Symmetric(fm1.A, :L)  # only the lower triangle is stored

8×8 Symmetric{Float64,BlockArrays.BlockArray{Float64,2,Array{AbstractArray{Float64,2},2},Tuple{BlockArrays.BlockedUnitRange{Array{Int64,1}},BlockArrays.BlockedUnitRange{Array{Int64,1}}}}} with indices 1:1:8×1:1:8:
    5.0     0.0     0.0     0.0     0.0     0.0  │      5.0  │   7525.0      
    0.0     5.0     0.0     0.0     0.0     0.0  │      5.0  │   7640.0      
    0.0     0.0     5.0     0.0     0.0     0.0  │      5.0  │   7820.0      
    0.0     0.0     0.0     5.0     0.0     0.0  │      5.0  │   7490.0      
    0.0     0.0     0.0     0.0     5.0     0.0  │      5.0  │   8000.0      
    0.0     0.0     0.0     0.0     0.0     5.0  │      5.0  │   7350.0      
 ────────────────────────────────────────────────┼───────────┼───────────────
    5.0     5.0     5.0     5.0     5.0     5.0  │     30.0  │  45825.0      
 ────────────────────────────────────────────────┼───────────┼───────────────
 7525.0  7640.0  7820.0  7490.0  8000.0  7350.0  │  45825.0  │      7.01129e7

In [10]:
A = Symmetric(MixedModels.copyltri(LowerTriangular(fm1.A)), :L)

8×8 Symmetric{Float64,Array{Float64,2}}:
    5.0     0.0     0.0     0.0     0.0     0.0      5.0   7525.0
    0.0     5.0     0.0     0.0     0.0     0.0      5.0   7640.0
    0.0     0.0     5.0     0.0     0.0     0.0      5.0   7820.0
    0.0     0.0     0.0     5.0     0.0     0.0      5.0   7490.0
    0.0     0.0     0.0     0.0     5.0     0.0      5.0   8000.0
    0.0     0.0     0.0     0.0     0.0     5.0      5.0   7350.0
    5.0     5.0     5.0     5.0     5.0     5.0     30.0  45825.0
 7525.0  7640.0  7820.0  7490.0  8000.0  7350.0  45825.0      7.01129e7

This matrix is `[Z X y]'[Z X y]` where `Z` is the model matrix for the random effects, `X` is the model matrix for the fixed-effects parameter, $\beta$, and `y` is the response.

In [11]:
ZXy = hcat(first(fm1.allterms), fm1.X, fm1.y)

30×8 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0  0.0  1.0  1545.0
 1.0  0.0  0.0  0.0  0.0  0.0  1.0  1440.0
 1.0  0.0  0.0  0.0  0.0  0.0  1.0  1440.0
 1.0  0.0  0.0  0.0  0.0  0.0  1.0  1520.0
 1.0  0.0  0.0  0.0  0.0  0.0  1.0  1580.0
 0.0  1.0  0.0  0.0  0.0  0.0  1.0  1540.0
 0.0  1.0  0.0  0.0  0.0  0.0  1.0  1555.0
 0.0  1.0  0.0  0.0  0.0  0.0  1.0  1490.0
 0.0  1.0  0.0  0.0  0.0  0.0  1.0  1560.0
 0.0  1.0  0.0  0.0  0.0  0.0  1.0  1495.0
 0.0  0.0  1.0  0.0  0.0  0.0  1.0  1595.0
 0.0  0.0  1.0  0.0  0.0  0.0  1.0  1550.0
 0.0  0.0  1.0  0.0  0.0  0.0  1.0  1605.0
 ⋮                        ⋮         
 0.0  0.0  0.0  1.0  0.0  0.0  1.0  1465.0
 0.0  0.0  0.0  1.0  0.0  0.0  1.0  1545.0
 0.0  0.0  0.0  0.0  1.0  0.0  1.0  1595.0
 0.0  0.0  0.0  0.0  1.0  0.0  1.0  1630.0
 0.0  0.0  0.0  0.0  1.0  0.0  1.0  1515.0
 0.0  0.0  0.0  0.0  1.0  0.0  1.0  1635.0
 0.0  0.0  0.0  0.0  1.0  0.0  1.0  1625.0
 0.0  0.0  0.0  0.0  0.0  1.0  1.0  1520.0
 0.0  0.0  0.0  0.0  0.0  1.0  1.0  1

In [12]:
A ≈ ZXy'ZXy

true

The parameter $θ$ generates a diagonal matrix $\Lambda$ that multiplies `Z` on the right.  For convenience extend $\Lambda$ on the diagonal with an identity matrix to multiply `ZXy`.

In [13]:
θ = only(fm1.θ)

0.7525806757718846

In [14]:
Λ = Diagonal(vcat(fill(θ, 6), ones(2)))

8×8 Diagonal{Float64,Array{Float64,1}}:
 0.752581   ⋅         ⋅         ⋅         ⋅         ⋅         ⋅    ⋅ 
  ⋅        0.752581   ⋅         ⋅         ⋅         ⋅         ⋅    ⋅ 
  ⋅         ⋅        0.752581   ⋅         ⋅         ⋅         ⋅    ⋅ 
  ⋅         ⋅         ⋅        0.752581   ⋅         ⋅         ⋅    ⋅ 
  ⋅         ⋅         ⋅         ⋅        0.752581   ⋅         ⋅    ⋅ 
  ⋅         ⋅         ⋅         ⋅         ⋅        0.752581   ⋅    ⋅ 
  ⋅         ⋅         ⋅         ⋅         ⋅         ⋅        1.0   ⋅ 
  ⋅         ⋅         ⋅         ⋅         ⋅         ⋅         ⋅   1.0

In [15]:
ZXyΛ = ZXy * Λ

30×8 Array{Float64,2}:
 0.752581  0.0       0.0       0.0       0.0       0.0       1.0  1545.0
 0.752581  0.0       0.0       0.0       0.0       0.0       1.0  1440.0
 0.752581  0.0       0.0       0.0       0.0       0.0       1.0  1440.0
 0.752581  0.0       0.0       0.0       0.0       0.0       1.0  1520.0
 0.752581  0.0       0.0       0.0       0.0       0.0       1.0  1580.0
 0.0       0.752581  0.0       0.0       0.0       0.0       1.0  1540.0
 0.0       0.752581  0.0       0.0       0.0       0.0       1.0  1555.0
 0.0       0.752581  0.0       0.0       0.0       0.0       1.0  1490.0
 0.0       0.752581  0.0       0.0       0.0       0.0       1.0  1560.0
 0.0       0.752581  0.0       0.0       0.0       0.0       1.0  1495.0
 0.0       0.0       0.752581  0.0       0.0       0.0       1.0  1595.0
 0.0       0.0       0.752581  0.0       0.0       0.0       1.0  1550.0
 0.0       0.0       0.752581  0.0       0.0       0.0       1.0  1605.0
 ⋮                          

In [16]:
ZXyΛ'ZXyΛ

8×8 Array{Float64,2}:
    2.83189     0.0         0.0         0.0      …      3.7629   5663.17
    0.0         2.83189     0.0         0.0             3.7629   5749.72
    0.0         0.0         2.83189     0.0             3.7629   5885.18
    0.0         0.0         0.0         2.83189         3.7629   5636.83
    0.0         0.0         0.0         0.0             3.7629   6020.65
    0.0         0.0         0.0         0.0      …      3.7629   5531.47
    3.7629      3.7629      3.7629      3.7629         30.0     45825.0
 5663.17     5749.72     5885.18     5636.83        45825.0         7.01129e7

Alternatively, this can be (and is) evaluated as

In [17]:
ΛAΛ = Λ'A*Λ

8×8 Array{Float64,2}:
    2.83189     0.0         0.0         0.0      …      3.7629   5663.17
    0.0         2.83189     0.0         0.0             3.7629   5749.72
    0.0         0.0         2.83189     0.0             3.7629   5885.18
    0.0         0.0         0.0         2.83189         3.7629   5636.83
    0.0         0.0         0.0         0.0             3.7629   6020.65
    0.0         0.0         0.0         0.0      …      3.7629   5531.47
    3.7629      3.7629      3.7629      3.7629         30.0     45825.0
 5663.17     5749.72     5885.18     5636.83        45825.0         7.01129e7

Finally an identity matrix is added to the upper left block, which is $Λ^\prime Z^\prime ZΛ$.  When working with the extended cross-product matrix we must extend the diagonal of this identity matrix with zeros in the positions of the `X` and `y` columns in `ZXy`.  To align with the notation of [https://arxiv.org/abs/1602.07527] this matrix is written $\Sigma$.

In [18]:
Σ = ΛAΛ + Diagonal(vcat(trues(6), falses(2)))

8×8 Array{Float64,2}:
    3.83189     0.0         0.0         0.0      …      3.7629   5663.17
    0.0         3.83189     0.0         0.0             3.7629   5749.72
    0.0         0.0         3.83189     0.0             3.7629   5885.18
    0.0         0.0         0.0         3.83189         3.7629   5636.83
    0.0         0.0         0.0         0.0             3.7629   6020.65
    0.0         0.0         0.0         0.0      …      3.7629   5531.47
    3.7629      3.7629      3.7629      3.7629         30.0     45825.0
 5663.17     5749.72     5885.18     5636.83        45825.0         7.01129e7

`L` is the lower Cholesky factor of this matrix

In [19]:
L*L' ≈ Σ

true

The objective, which is negative twice the log-likelihood, is $\log\left(\det\left(\Lambda^\prime Z^\prime Z \Lambda + I\right)\right) + n\left(1+\log\left(\frac{2\pi\ell_{yy}^2}{n}\right)\right)$, where $\ell_{yy}$ is the lower-right element of `L`, which happens to be the square root of the penalized sum of squared residuals, and $n$ is the number of observations.  $\log\left(\det\left(\Lambda^\prime Z^\prime Z \Lambda + I\right)\right)$ is evaluated as twice the sum of the logarithms of the diagonal elements of the upper $6\times6$ submatrix of `L`.

In [20]:
logdet(fm1)

8.060146368533138

In [21]:
2sum(log, diag(L)[1:6])

8.060146368533138

The second term is evaluated from the estimate of the squared dispersion

In [22]:
log(dispersion(fm1, true))

7.804353384010887

In [23]:
n = length(fm1.y)

30

In [24]:
MixedModels.log2π

log2π = 1.8378770664093...

In [25]:
logdet(fm1) + n*(1 + MixedModels.log2π + log(dispersion(fm1, true)))

327.3270598811401

In [26]:
objective(fm1)

327.3270598811401

## The gradient of the objective

**Automatic differentiation** (AD) describes a collection of techniques for evaluating the gradient of an expression directly from the expression itself by application of the [chain rule](https://en.wikipedia.org/wiki/Chain_rule).  The evaluation of the objective, $f$, as a function of $\theta$, can be decomposed as $\theta\rightarrow\Lambda\rightarrow\Sigma\rightarrow L\rightarrow f$.  Application of the *forward rule* involves composing the forward sensitivities, usually written with a dot diacritical.

Thus $\dot{\Lambda}=\frac{d\Lambda}{d\theta}$ from which $\frac{d\Sigma}{d\theta}=\dot{\Sigma}=\dot{\Lambda}^\prime A \Lambda+\Lambda^\prime A \dot{\Lambda}$ can be evaluated. [https://arxiv.org/abs/1602.07527] provides methods to evaluate $\frac{dL}{d\theta}=\dot{L}$ from $\dot{\Sigma}$ and $L$.  Finally, given $\dot{L}$ and $L$, the gradient $\dot{f}=\frac{d\,f}{d\theta}$ can be evaluated.

Going through this, step-by-step,

In [27]:
Λ̇ = Diagonal(vcat(trues(6), falses(2)))

8×8 Diagonal{Bool,BitArray{1}}:
 1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  0  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  0

In [28]:
Σ̇ = Λ̇'A*Λ+Λ'A*Λ̇

8×8 Array{Float64,2}:
    7.52581     0.0         0.0      …     0.0         0.0      5.0  7525.0
    0.0         7.52581     0.0            0.0         0.0      5.0  7640.0
    0.0         0.0         7.52581        0.0         0.0      5.0  7820.0
    0.0         0.0         0.0            0.0         0.0      5.0  7490.0
    0.0         0.0         0.0            7.52581     0.0      5.0  8000.0
    0.0         0.0         0.0      …     0.0         7.52581  5.0  7350.0
    5.0         5.0         5.0            5.0         5.0      0.0     0.0
 7525.0      7640.0      7820.0         8000.0      7350.0      0.0     0.0

The function `MixedModels.chol_unblocked_fwd`, an almost literal transcription of the pseudo-code on p. 6 of [https://arxiv.org/abs/1602.07527], provides

In [29]:
L̇ = MixedModels.chol_unblocked_fwd(Matrix(LowerTriangular(Σ̇)), L)

8×8 Array{Float64,2}:
    1.92228      0.0          0.0       …    0.0           0.0        0.0
    0.0          1.92228      0.0            0.0           0.0        0.0
    0.0          0.0          1.92228        0.0           0.0        0.0
    0.0          0.0          0.0            0.0           0.0        0.0
    0.0          0.0          0.0            0.0           0.0        0.0
    0.0          0.0          0.0       …    1.92228       0.0        0.0
    0.666578     0.666578     0.666578       0.666578     -2.74767    0.0
 1003.2       1018.53      1042.53         979.869     -4197.06     -53.2592

In [30]:
ḟ = 2dot(diag(L̇), vcat(ones(6), 0, n) ./ diag(L))

-3.063889675303244e-7

This derivative is very close to zero, as it should be at a estimate $\widehat{\theta}$ which is not on the boundary.

## Models with multiple simple, scalar random-effects terms

Consider a model with random effects for both `sample` and `plate` fit to the `penicillin` data.

In [31]:
penicillin = dataset(:penicillin)
describe(penicillin)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Nothing,DataType
1,plate,,a,,x,24,,String
2,sample,,A,,F,6,,String
3,diameter,22.9722,18,23.0,27,,,Int8


In [32]:
fm2 = fit(MixedModel, @formula(diameter ~ 1 + (1|plate) + (1|sample)), penicillin)

Linear mixed model fit by maximum likelihood
 diameter ~ 1 + (1 | plate) + (1 | sample)
   logLik   -2 logLik     AIC        BIC    
 -166.09417  332.18835  340.18835  352.06760

Variance components:
            Column    Variance   Std.Dev. 
plate    (Intercept)  0.71497949 0.8455646
sample   (Intercept)  3.13519360 1.7706478
Residual              0.30242640 0.5499331
 Number of obs: 144; levels of grouping factors: 24, 6

  Fixed-effects parameters:
──────────────────────────────────────────────────
             Estimate  Std.Error  z value  P(>|z|)
──────────────────────────────────────────────────
(Intercept)   22.9722   0.744596    30.85   <1e-99
──────────────────────────────────────────────────

The $\theta$ parameter for this model is two-dimensional

In [33]:
θ = fm2.θ

2-element Array{Float64,1}:
 1.5375772433917159
 3.219751343843134

In [34]:
L = MixedModels.copyltri(fm2.L)

32×32 Array{Float64,2}:
  3.89678   0.0       0.0       0.0      …   0.0       0.0       0.0
  0.0       3.89678   0.0       0.0          0.0       0.0       0.0
  0.0       0.0       3.89678   0.0          0.0       0.0       0.0
  0.0       0.0       0.0       3.89678      0.0       0.0       0.0
  0.0       0.0       0.0       0.0          0.0       0.0       0.0
  0.0       0.0       0.0       0.0      …   0.0       0.0       0.0
  0.0       0.0       0.0       0.0          0.0       0.0       0.0
  0.0       0.0       0.0       0.0          0.0       0.0       0.0
  0.0       0.0       0.0       0.0          0.0       0.0       0.0
  0.0       0.0       0.0       0.0          0.0       0.0       0.0
  0.0       0.0       0.0       0.0      …   0.0       0.0       0.0
  0.0       0.0       0.0       0.0          0.0       0.0       0.0
  0.0       0.0       0.0       0.0          0.0       0.0       0.0
  ⋮                                      ⋱             ⋮         
  0.0       0

In [35]:
Λ = Diagonal(vcat(fill(first(θ), 24), fill(last(θ), 6), ones(2)))

32×32 Diagonal{Float64,Array{Float64,1}}:
 1.53758   ⋅        ⋅        ⋅       …   ⋅        ⋅        ⋅        ⋅    ⋅ 
  ⋅       1.53758   ⋅        ⋅           ⋅        ⋅        ⋅        ⋅    ⋅ 
  ⋅        ⋅       1.53758   ⋅           ⋅        ⋅        ⋅        ⋅    ⋅ 
  ⋅        ⋅        ⋅       1.53758      ⋅        ⋅        ⋅        ⋅    ⋅ 
  ⋅        ⋅        ⋅        ⋅           ⋅        ⋅        ⋅        ⋅    ⋅ 
  ⋅        ⋅        ⋅        ⋅       …   ⋅        ⋅        ⋅        ⋅    ⋅ 
  ⋅        ⋅        ⋅        ⋅           ⋅        ⋅        ⋅        ⋅    ⋅ 
  ⋅        ⋅        ⋅        ⋅           ⋅        ⋅        ⋅        ⋅    ⋅ 
  ⋅        ⋅        ⋅        ⋅           ⋅        ⋅        ⋅        ⋅    ⋅ 
  ⋅        ⋅        ⋅        ⋅           ⋅        ⋅        ⋅        ⋅    ⋅ 
  ⋅        ⋅        ⋅        ⋅       …   ⋅        ⋅        ⋅        ⋅    ⋅ 
  ⋅        ⋅        ⋅        ⋅           ⋅        ⋅        ⋅        ⋅    ⋅ 
  ⋅        ⋅        ⋅        ⋅           ⋅    

In [36]:
A = Matrix(Symmetric(MixedModels.copyltri(fm2.A), :L))

32×32 Array{Float64,2}:
   6.0    0.0    0.0    0.0    0.0    0.0  …    1.0    1.0     6.0    143.0
   0.0    6.0    0.0    0.0    0.0    0.0       1.0    1.0     6.0    143.0
   0.0    0.0    6.0    0.0    0.0    0.0       1.0    1.0     6.0    139.0
   0.0    0.0    0.0    6.0    0.0    0.0       1.0    1.0     6.0    140.0
   0.0    0.0    0.0    0.0    6.0    0.0       1.0    1.0     6.0    138.0
   0.0    0.0    0.0    0.0    0.0    6.0  …    1.0    1.0     6.0    135.0
   0.0    0.0    0.0    0.0    0.0    0.0       1.0    1.0     6.0    129.0
   0.0    0.0    0.0    0.0    0.0    0.0       1.0    1.0     6.0    143.0
   0.0    0.0    0.0    0.0    0.0    0.0       1.0    1.0     6.0    133.0
   0.0    0.0    0.0    0.0    0.0    0.0       1.0    1.0     6.0    133.0
   0.0    0.0    0.0    0.0    0.0    0.0  …    1.0    1.0     6.0    144.0
   0.0    0.0    0.0    0.0    0.0    0.0       1.0    1.0     6.0    141.0
   0.0    0.0    0.0    0.0    0.0    0.0       1.0    1.0     6

The first element of the gradient is evaluated from

In [37]:
Λ̇ = Diagonal(vcat(trues(24), falses(8)))
Σ̇ = Λ̇'A*Λ+Λ'A*Λ̇

32×32 Array{Float64,2}:
  18.4509     0.0        0.0        0.0      …  3.21975  3.21975  6.0  143.0
   0.0       18.4509     0.0        0.0         3.21975  3.21975  6.0  143.0
   0.0        0.0       18.4509     0.0         3.21975  3.21975  6.0  139.0
   0.0        0.0        0.0       18.4509      3.21975  3.21975  6.0  140.0
   0.0        0.0        0.0        0.0         3.21975  3.21975  6.0  138.0
   0.0        0.0        0.0        0.0      …  3.21975  3.21975  6.0  135.0
   0.0        0.0        0.0        0.0         3.21975  3.21975  6.0  129.0
   0.0        0.0        0.0        0.0         3.21975  3.21975  6.0  143.0
   0.0        0.0        0.0        0.0         3.21975  3.21975  6.0  133.0
   0.0        0.0        0.0        0.0         3.21975  3.21975  6.0  133.0
   0.0        0.0        0.0        0.0      …  3.21975  3.21975  6.0  144.0
   0.0        0.0        0.0        0.0         3.21975  3.21975  6.0  141.0
   0.0        0.0        0.0        0.0         3.21

In [38]:
L̇ = MixedModels.chol_unblocked_fwd(Matrix(LowerTriangular(Σ̇)), L)

32×32 Array{Float64,2}:
 2.36746    0.0        0.0        …    0.0       0.0         0.0
 0.0        2.36746    0.0             0.0       0.0         0.0
 0.0        0.0        2.36746         0.0       0.0         0.0
 0.0        0.0        0.0             0.0       0.0         0.0
 0.0        0.0        0.0             0.0       0.0         0.0
 0.0        0.0        0.0        …    0.0       0.0         0.0
 0.0        0.0        0.0             0.0       0.0         0.0
 0.0        0.0        0.0             0.0       0.0         0.0
 0.0        0.0        0.0             0.0       0.0         0.0
 0.0        0.0        0.0             0.0       0.0         0.0
 0.0        0.0        0.0        …    0.0       0.0         0.0
 0.0        0.0        0.0             0.0       0.0         0.0
 0.0        0.0        0.0             0.0       0.0         0.0
 ⋮                                ⋱              ⋮          
 0.0        0.0        0.0        …    0.0       0.0         0.0
 0.0 

In [39]:
ḟ = 2dot(diag(L̇), vcat(ones(30), 0, 144) ./ diag(L))

-0.00046524874183973

The second element of the gradient is evaluated from

In [40]:
Λ̇ = Diagonal(vcat(falses(24), trues(6), falses(2)))
Σ̇ = Λ̇'A*Λ + Λ'A*Λ̇

32×32 Array{Float64,2}:
 0.0      0.0      0.0      0.0      …    1.53758    1.53758   0.0    0.0
 0.0      0.0      0.0      0.0           1.53758    1.53758   0.0    0.0
 0.0      0.0      0.0      0.0           1.53758    1.53758   0.0    0.0
 0.0      0.0      0.0      0.0           1.53758    1.53758   0.0    0.0
 0.0      0.0      0.0      0.0           1.53758    1.53758   0.0    0.0
 0.0      0.0      0.0      0.0      …    1.53758    1.53758   0.0    0.0
 0.0      0.0      0.0      0.0           1.53758    1.53758   0.0    0.0
 0.0      0.0      0.0      0.0           1.53758    1.53758   0.0    0.0
 0.0      0.0      0.0      0.0           1.53758    1.53758   0.0    0.0
 0.0      0.0      0.0      0.0           1.53758    1.53758   0.0    0.0
 0.0      0.0      0.0      0.0      …    1.53758    1.53758   0.0    0.0
 0.0      0.0      0.0      0.0           1.53758    1.53758   0.0    0.0
 0.0      0.0      0.0      0.0           1.53758    1.53758   0.0    0.0
 ⋮            

In [41]:
L̇ = MixedModels.chol_unblocked_fwd(Matrix(LowerTriangular(Σ̇)), L)

32×32 Array{Float64,2}:
 0.0       0.0       0.0       0.0       …  0.0         0.0        0.0
 0.0       0.0       0.0       0.0          0.0         0.0        0.0
 0.0       0.0       0.0       0.0          0.0         0.0        0.0
 0.0       0.0       0.0       0.0          0.0         0.0        0.0
 0.0       0.0       0.0       0.0          0.0         0.0        0.0
 0.0       0.0       0.0       0.0       …  0.0         0.0        0.0
 0.0       0.0       0.0       0.0          0.0         0.0        0.0
 0.0       0.0       0.0       0.0          0.0         0.0        0.0
 0.0       0.0       0.0       0.0          0.0         0.0        0.0
 0.0       0.0       0.0       0.0          0.0         0.0        0.0
 0.0       0.0       0.0       0.0       …  0.0         0.0        0.0
 0.0       0.0       0.0       0.0          0.0         0.0        0.0
 0.0       0.0       0.0       0.0          0.0         0.0        0.0
 ⋮                                       ⋱           

In [42]:
ḟ = 2dot(diag(L̇), vcat(ones(30), 0, 144) ./ diag(L))

6.920586637315651e-6

## Models with vector-valued random effects

Based on our favorite model fit to the `sleepstudy` data

In [43]:
sleepstudy = dataset(:sleepstudy)
fm3 = fit(MixedModel, @formula(reaction ~ 1 + days + (1+days|subj)), sleepstudy)

Linear mixed model fit by maximum likelihood
 reaction ~ 1 + days + (1 + days | subj)
   logLik   -2 logLik     AIC        BIC    
 -875.96967 1751.93934 1763.93934 1783.09709

Variance components:
            Column    Variance  Std.Dev.   Corr.
subj     (Intercept)  565.51068 23.780469
         days          32.68212  5.716828 +0.08
Residual              654.94145 25.591824
 Number of obs: 180; levels of grouping factors: 18

  Fixed-effects parameters:
──────────────────────────────────────────────────
             Estimate  Std.Error  z value  P(>|z|)
──────────────────────────────────────────────────
(Intercept)  251.405     6.63226    37.91   <1e-99
days          10.4673    1.50224     6.97   <1e-11
──────────────────────────────────────────────────

In [44]:
L = MixedModels.copyltri(fm3.L)

39×39 Array{Float64,2}:
    3.35381    0.0         0.0      …     0.0        0.0       0.0
    3.11966    2.32279     0.0            0.0        0.0       0.0
    0.0        0.0         3.35381        0.0        0.0       0.0
    0.0        0.0         3.11966        0.0        0.0       0.0
    0.0        0.0         0.0            0.0        0.0       0.0
    0.0        0.0         0.0      …     0.0        0.0       0.0
    0.0        0.0         0.0            0.0        0.0       0.0
    0.0        0.0         0.0            0.0        0.0       0.0
    0.0        0.0         0.0            0.0        0.0       0.0
    0.0        0.0         0.0            0.0        0.0       0.0
    0.0        0.0         0.0      …     0.0        0.0       0.0
    0.0        0.0         0.0            0.0        0.0       0.0
    0.0        0.0         0.0            0.0        0.0       0.0
    ⋮                               ⋱                        
    0.0        0.0         0.0            0

In [45]:
A = Matrix(Symmetric(MixedModels.copyltri(fm3.A), :L))

39×39 Array{Float64,2}:
   10.0      45.0     0.0      0.0   …     10.0      45.0    3421.34
   45.0     285.0     0.0      0.0         45.0     285.0   17191.6
    0.0       0.0    10.0     45.0         10.0      45.0    2152.33
    0.0       0.0    45.0    285.0         45.0     285.0    9872.08
    0.0       0.0     0.0      0.0         10.0      45.0    2310.01
    0.0       0.0     0.0      0.0   …     45.0     285.0   10899.5
    0.0       0.0     0.0      0.0         10.0      45.0    3032.21
    0.0       0.0     0.0      0.0         45.0     285.0   13893.1
    0.0       0.0     0.0      0.0         10.0      45.0    3094.36
    0.0       0.0     0.0      0.0         45.0     285.0   14359.1
    0.0       0.0     0.0      0.0   …     10.0      45.0    3073.02
    0.0       0.0     0.0      0.0         45.0     285.0   14617.9
    0.0       0.0     0.0      0.0         10.0      45.0    3161.58
    ⋮                                ⋱                     
    0.0       0.0     0.

For the first element of

In [46]:
θ = fm3.θ

3-element Array{Float64,1}:
 0.9292213238823973
 0.018168399088001212
 0.22264486437568012

In [47]:
using BlockDiagonals

In [48]:
Λ = BlockDiagonal([Matrix(BlockDiagonal(fill(first(fm3.λ).data, 18))), Diagonal(ones(3))])

39×39 BlockDiagonal{Float64,AbstractArray{Float64,2}}:
 0.929221   0.0       0.0        …  0.0        0.0       0.0  0.0  0.0
 0.0181684  0.222645  0.0           0.0        0.0       0.0  0.0  0.0
 0.0        0.0       0.929221      0.0        0.0       0.0  0.0  0.0
 0.0        0.0       0.0181684     0.0        0.0       0.0  0.0  0.0
 0.0        0.0       0.0           0.0        0.0       0.0  0.0  0.0
 0.0        0.0       0.0        …  0.0        0.0       0.0  0.0  0.0
 0.0        0.0       0.0           0.0        0.0       0.0  0.0  0.0
 0.0        0.0       0.0           0.0        0.0       0.0  0.0  0.0
 0.0        0.0       0.0           0.0        0.0       0.0  0.0  0.0
 0.0        0.0       0.0           0.0        0.0       0.0  0.0  0.0
 0.0        0.0       0.0        …  0.0        0.0       0.0  0.0  0.0
 0.0        0.0       0.0           0.0        0.0       0.0  0.0  0.0
 0.0        0.0       0.0           0.0        0.0       0.0  0.0  0.0
 ⋮                    

In [49]:
Λ̇ = Diagonal(vcat(repeat([true, false], outer=18), falses(3)))
Σ̇ = Λ̇'A*Λ + Λ'A*Λ̇

39×39 Array{Float64,2}:
   20.2196  10.019     0.0      0.0    …   0.0    10.0  45.0  3421.34
   10.019    0.0       0.0      0.0        0.0     0.0   0.0     0.0
    0.0      0.0      20.2196  10.019      0.0    10.0  45.0  2152.33
    0.0      0.0      10.019    0.0        0.0     0.0   0.0     0.0
    0.0      0.0       0.0      0.0        0.0    10.0  45.0  2310.01
    0.0      0.0       0.0      0.0    …   0.0     0.0   0.0     0.0
    0.0      0.0       0.0      0.0        0.0    10.0  45.0  3032.21
    0.0      0.0       0.0      0.0        0.0     0.0   0.0     0.0
    0.0      0.0       0.0      0.0        0.0    10.0  45.0  3094.36
    0.0      0.0       0.0      0.0        0.0     0.0   0.0     0.0
    0.0      0.0       0.0      0.0    …   0.0    10.0  45.0  3073.02
    0.0      0.0       0.0      0.0        0.0     0.0   0.0     0.0
    0.0      0.0       0.0      0.0        0.0    10.0  45.0  3161.58
    ⋮                                  ⋱   ⋮                  
    0.0  

In [50]:
L̇ = MixedModels.chol_unblocked_fwd(Matrix(LowerTriangular(Σ̇)), L)

39×39 Array{Float64,2}:
  3.01442      0.0        0.0       …     0.0       0.0            0.0
  0.183386    -0.2463     0.0             0.0       0.0            0.0
  0.0          0.0        3.01442         0.0       0.0            0.0
  0.0          0.0        0.183386        0.0       0.0            0.0
  0.0          0.0        0.0             0.0       0.0            0.0
  0.0          0.0        0.0       …     0.0       0.0            0.0
  0.0          0.0        0.0             0.0       0.0            0.0
  0.0          0.0        0.0             0.0       0.0            0.0
  0.0          0.0        0.0             0.0       0.0            0.0
  0.0          0.0        0.0             0.0       0.0            0.0
  0.0          0.0        0.0       …     0.0       0.0            0.0
  0.0          0.0        0.0             0.0       0.0            0.0
  0.0          0.0        0.0             0.0       0.0            0.0
  ⋮                                 ⋱                

In [51]:
ḟ = 2dot(diag(L̇), vcat(ones(36), zeros(2), 180) ./ diag(L))

-7.670801159065377e-5

For the last element of $\theta$ the gradient value is calculated from

In [52]:
Λ̇ =  Diagonal(vcat(repeat([false, true], outer=18), falses(3)))
Σ̇ = Λ̇'A*Λ + Λ'A*Λ̇

39×39 Array{Float64,2}:
  0.0       46.993   0.0       0.0    …      0.0     0.0    0.0      0.0
 46.993    126.908   0.0       0.0           0.0    45.0  285.0  17191.6
  0.0        0.0     0.0      46.993         0.0     0.0    0.0      0.0
  0.0        0.0    46.993   126.908         0.0    45.0  285.0   9872.08
  0.0        0.0     0.0       0.0           0.0     0.0    0.0      0.0
  0.0        0.0     0.0       0.0    …      0.0    45.0  285.0  10899.5
  0.0        0.0     0.0       0.0           0.0     0.0    0.0      0.0
  0.0        0.0     0.0       0.0           0.0    45.0  285.0  13893.1
  0.0        0.0     0.0       0.0           0.0     0.0    0.0      0.0
  0.0        0.0     0.0       0.0           0.0    45.0  285.0  14359.1
  0.0        0.0     0.0       0.0    …      0.0     0.0    0.0      0.0
  0.0        0.0     0.0       0.0           0.0    45.0  285.0  14617.9
  0.0        0.0     0.0       0.0           0.0     0.0    0.0      0.0
  ⋮                       

In [53]:
L̇ = MixedModels.chol_unblocked_fwd(Matrix(LowerTriangular(Σ̇)), L)

39×39 Array{Float64,2}:
  0.0       0.0        0.0      0.0       …     0.0          0.0        0.0
 14.0118    8.49909    0.0      0.0             0.0          0.0        0.0
  0.0       0.0        0.0      0.0             0.0          0.0        0.0
  0.0       0.0       14.0118   8.49909         0.0          0.0        0.0
  0.0       0.0        0.0      0.0             0.0          0.0        0.0
  0.0       0.0        0.0      0.0       …     0.0          0.0        0.0
  0.0       0.0        0.0      0.0             0.0          0.0        0.0
  0.0       0.0        0.0      0.0             0.0          0.0        0.0
  0.0       0.0        0.0      0.0             0.0          0.0        0.0
  0.0       0.0        0.0      0.0             0.0          0.0        0.0
  0.0       0.0        0.0      0.0       …     0.0          0.0        0.0
  0.0       0.0        0.0      0.0             0.0          0.0        0.0
  0.0       0.0        0.0      0.0             0.0          0.0

In [54]:
ḟ = 2dot(diag(L̇), vcat(ones(36), zeros(2), 180) ./ diag(L))

7.034703170916146e-5

Finally, for the term defining the correlation

In [55]:
Λ̇ = Bidiagonal(fill(false, 39), vcat(repeat([true, false], 18), falses(2)), :L)

39×39 Bidiagonal{Bool,Array{Bool,1}}:
 0  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  …  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 1  0  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  0  0  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  1  0  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  0  0  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  1  0  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  …  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  0  0  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  0  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  0  0  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  0  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  0  0  ⋅  ⋅  …  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  0  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅

In [56]:
Σ̇ = Λ̇'A*Λ + Λ'A*Λ̇

39×39 Array{Float64,2}:
    93.9859  63.4538     0.0      0.0     …   0.0     45.0  285.0  17191.6
    63.4538   0.0        0.0      0.0         0.0      0.0    0.0      0.0
     0.0      0.0       93.9859  63.4538      0.0     45.0  285.0   9872.08
     0.0      0.0       63.4538   0.0         0.0      0.0    0.0      0.0
     0.0      0.0        0.0      0.0         0.0     45.0  285.0  10899.5
     0.0      0.0        0.0      0.0     …   0.0      0.0    0.0      0.0
     0.0      0.0        0.0      0.0         0.0     45.0  285.0  13893.1
     0.0      0.0        0.0      0.0         0.0      0.0    0.0      0.0
     0.0      0.0        0.0      0.0         0.0     45.0  285.0  14359.1
     0.0      0.0        0.0      0.0         0.0      0.0    0.0      0.0
     0.0      0.0        0.0      0.0     …   0.0     45.0  285.0  14617.9
     0.0      0.0        0.0      0.0         0.0      0.0    0.0      0.0
     0.0      0.0        0.0      0.0         0.0     45.0  285.0  14981.3


In [57]:
L̇ = MixedModels.chol_unblocked_fwd(Matrix(LowerTriangular(Σ̇)), L)

39×39 Array{Float64,2}:
  14.0118        0.0        0.0       …      0.0        0.0        0.0
   5.88634      -7.90573    0.0              0.0        0.0        0.0
   0.0           0.0       14.0118           0.0        0.0        0.0
   0.0           0.0        5.88634          0.0        0.0        0.0
   0.0           0.0        0.0              0.0        0.0        0.0
   0.0           0.0        0.0       …      0.0        0.0        0.0
   0.0           0.0        0.0              0.0        0.0        0.0
   0.0           0.0        0.0              0.0        0.0        0.0
   0.0           0.0        0.0              0.0        0.0        0.0
   0.0           0.0        0.0              0.0        0.0        0.0
   0.0           0.0        0.0       …      0.0        0.0        0.0
   0.0           0.0        0.0              0.0        0.0        0.0
   0.0           0.0        0.0              0.0        0.0        0.0
   ⋮                                  ⋱              

In [58]:
ḟ = 2dot(diag(L̇), vcat(ones(36), zeros(2), 180) ./ diag(L))

0.0010090706624659163

## Forward mode versus reverse mode

A Julia implementation of the reverse-mode automatic differentiation algorithm given in [https://arxiv.org/abs/1602.07527] is available in the [Nabla.jl](https://github.com/invenia/Nable.jl) package.  In fact, the forward-mode AD implementation in the [MixedModels.jl](https://github.com/JuliaStats/MixedModels.jl) package is directly patterned on the `Nabla.jl` implementation of reverse-mode AD.

In [59]:
?MixedModels.chol_unblocked_fwd!

```
chol_unblocked_fwd!(Σ̇::AbstractMatrix, L::AbstractMatrix)
```

Overwrite the sensitivities, Σ̇, by the sensitivities, L̇, of the lower Cholesky factor, L

This is the scalar, unblocked version of the forward mode differentiation of the Cholesky factor given at the bottom of p. 6 in the reference.  Only the lower triangle of Σ̇ is referenced and overwritten.

Note how close this code is to the pseudo-code in the reference - much closer than the Python code in the appendix.


In [60]:
using Nabla

In [61]:
?Nabla.chol_unblocked_rev!

```
chol_unblocked_rev!(
    Ā::AbstractMatrix{T},
    L::AbstractMatrix{T},
    upper::Bool
) where T<:Real
```

Compute the reverse-mode sensitivities of the Cholesky factorisation in an unblocked manner. If `upper` is `false`, then the sensitivites computed from and stored in the lower triangle of `Ā` and `L` respectively. If `upper` is `true` then they are computed and stored in the upper triangles. If at input `upper` is `false` and `tril(Ā) = L̄`, at output `tril(Ā) = tril(Σ̄)`, where `Σ = LLᵀ`. Analogously, if at input `upper` is `true` and `triu(Ā) = triu(Ū)`, at output `triu(Ā) = triu(Σ̄)` where `Σ = UᵀU`.


The big advantage of forward-mode AD for this calculation is that the gradient evaluation preserves the sparsity patterns of $\mathbf{L}$ in $\dot{\mathbf{L}}$.  Recall that it is the sparsity of $\mathbf{L}$ that makes the whole calculation feasible for complex models fit to large data sets.  The disadvantage of forward-mode relative to reverse mode is that the calculations, especially evaluating $\dot{\mathbf{L}}$ from $\dot{\mathbf{\Sigma}}$ must be repeated for each element of the gradient vector.  In reverse mode the sensitivities, written $\bar{\mathbf{L}}$, etc., are "pulled back" from $f$ to $\mathbf{L}$ to $\mathbf{\Sigma}$ to $\mathbf{\Lambda}$ and finally to $\mathbf{\theta}$.  There is only one calculation at each step until the last.  Unfortunately the sparsity is lost at the first step.

In [62]:
L̄ = Matrix(Diagonal(2vcat(inv.(diag(L)[1:36]), zeros(2), 180.0/last(L))))

39×39 Array{Float64,2}:
 0.596337  0.0       0.0       0.0       …  0.0       0.0  0.0  0.0
 0.0       0.861032  0.0       0.0          0.0       0.0  0.0  0.0
 0.0       0.0       0.596337  0.0          0.0       0.0  0.0  0.0
 0.0       0.0       0.0       0.861032     0.0       0.0  0.0  0.0
 0.0       0.0       0.0       0.0          0.0       0.0  0.0  0.0
 0.0       0.0       0.0       0.0       …  0.0       0.0  0.0  0.0
 0.0       0.0       0.0       0.0          0.0       0.0  0.0  0.0
 0.0       0.0       0.0       0.0          0.0       0.0  0.0  0.0
 0.0       0.0       0.0       0.0          0.0       0.0  0.0  0.0
 0.0       0.0       0.0       0.0          0.0       0.0  0.0  0.0
 0.0       0.0       0.0       0.0       …  0.0       0.0  0.0  0.0
 0.0       0.0       0.0       0.0          0.0       0.0  0.0  0.0
 0.0       0.0       0.0       0.0          0.0       0.0  0.0  0.0
 ⋮                                       ⋱  ⋮                   
 0.0       0.0       0.0   

In [63]:
Σ̄ = Nabla.chol_unblocked_rev(L̄, L, false)

39×39 Array{Float64,2}:
  0.263293     0.0          0.0       …   0.0        0.0       0.0
  0.0301036    2.69162      0.0           0.0        0.0       0.0
 -0.398823    -5.33224      3.08543       0.0        0.0       0.0
 -0.326724    -4.36828      4.30207       0.0        0.0       0.0
 -0.382736    -5.11716      5.44352       0.0        0.0       0.0
 -0.197918    -2.64615      2.81492   …   0.0        0.0       0.0
  0.227374     3.03998     -3.23386       0.0        0.0       0.0
 -0.212182    -2.83686      3.01779       0.0        0.0       0.0
  0.214604     2.86925     -3.05224       0.0        0.0       0.0
 -0.139892    -1.87036      1.98964       0.0        0.0       0.0
  0.0877896    1.17374     -1.2486    …   0.0        0.0       0.0
 -0.0169394   -0.226479     0.240923      0.0        0.0       0.0
  0.163737     2.18915     -2.32877       0.0        0.0       0.0
  ⋮                                   ⋱                        
  0.131275     1.75514     -1.86708      

It seems more reasonable to repeat the forward mode evaluations using the blocked structure already in place than to have the problem size blow up by replacing sparse, blocked operations by dense operations. However, this is something that should be verified through actual implementation and benchmarking.